In [58]:
import numpy as np

class HMM():
    def __init__(self, transition, mu,sigma, obs,T,pi):
        self.transition = transition #shape 4x4
        self.state=[0,1,2,3] #a modifier 
        self.pi=pi
        self.mu = mu
        self.sigma = sigma
        self.obs=obs
        self.T=T
        self.q=np.zeros(T)

    
    def  multivariate_gaussian(self, xi):
        """ Computes the vector of the probabilities p(u_t | q_t=i )"""
        probs = np.zeros(4)
        mu = self.mu
        sigmas = self.sigma
        pi = self.pi
        inv_sigmas = [np.linalg.inv(sigmas[j]) for j in range(4)]
        det_sigmas = [np.linalg.det(sigmas[j]) for j in range(4)]
        for j in range(4):
            x_m=(xi-mu[j]).reshape(2,1)
            probs[j] =pi[j]*math.exp(-(xi - mu[j]).T.dot(inv_sigmas[j]).dot(xi - mu[j]) / 2)
            probs[j] /= (2 * math.pi * math.sqrt(det_sigmas[j]))

        probs = probs / np.sum(probs)
        return probs
    
        
    def alpha_(self):
        alphas=[[] for t in range(self.T)]
        alphas[0]=self.pi
        print(self.pi,'pi')
        for index in range(1,self.T):
            probs=self.multivariate_gaussian(self.obs[index])
            result=np.zeros(4)
            for q in self.state:
                print('probq',probs[q])
                print('transition*alpha',np.sum(self.transition[q,:].dot(alphas[index-1])))
                result[q]=probs[q]*np.sum(self.transition[q,:].dot(alphas[index-1]))
                print('resul',result[q])
            alphas[index]=result
        return alphas 

#     def alpha_(self,index,probs): #backward
#         result=np.zeros(4)
#         if index==0:
#             return self.pi
#         else:
#             probs=self.multivariate_gaussian(self.obs[index-1])
#             alpha=self.alpha_(index-1,probs)
#             for q in self.state:
#                 result[q]= probs[q]*np.sum(self.transition[q,:].dot(alpha))
#             return result
        
    def beta_(self): #forwoard
        betas=[[] for t in range(self.T)]
        betas[(T-1)]=1
        for index in range((self.T-2),-1,-1):
            probs=self.multivariate_gaussian(self.obs[index])
            result=np.zeros(4)
            for q in self.state:
                result[q]=np.sum(probs[q]*self.transition[:,q].dot(betas[index+1]))
            betas[index]=result
        return betas
                
#         result=np.zeros(4)
#         if index == T:
#             return 1
#         else: 
#             beta = self.beta_(index+1,probs)
#             for q in self.state:
#                 result[q]=np.sum(probs[q]*self.transition[:,q].dot(beta))  
#             return result 
        
    def estimate_transition(self,alphas,betas):
        for k in range(4):
            for l in range(4):
                res=0
                denominateur=0
                for index in range(self.T):
                    probs=self.multivariate_gaussian(self.obs[index])
                    res+=self.transition[k,l]*alphas[index][l]*betas[index][k]*probs[k]
                    denominateur+=np.sum(alphas[index].dot(betas[index]))
                self.transition[k,l]=res/denominateur
        print('transision',self.transition)
        return self.transition
    
    def estimate_mu(self,alphas,betas):
        for k in range(4):
            res=0
            for index in range(self.T):
                res = res + alphas[index][k]*betas[index][k]*(self.obs[index])/np.sum([alphas[index][k]*betas[index][k] for index in range(T)])
            self.mu[k]=res
        print('selfmu',self.mu)
        return self.mu
    def estimate_sigma(self,alphas,betas):                         
        for k in range(4):
            res=0
            for index in range(self.T):
                x_u=self.obs[index]-self.mu[k]
                x_u=x_u.reshape(2,1)
                res = res+ alphas[index][k]*betas[index][k]*(x_u).dot(x_u.T)/np.sum([alphas[index][k]*betas[index][k] for index in range(T)])
            self.sigma[k]=res
        print('selfsigmas',self.sigma)
        return self.sigma
    
    def estimate_multinomial(self,alphas,betas):
        for k in range(4):
            self.pi[k]=alphas[0][k]*betas[0][k]/np.sum(alphas[0].dot(betas[0]))
        return self.pi
    
        
    def fit(self): 
        for j in range(10):
            #forward,backward (E-step)
            alphas=self.alpha_()
            betas=self.beta_()
            for index in range(self.T):
                self.q[index]=np.argmax(alphas[index] * betas[index] /np.sum(alphas[index].dot(betas[index])))
        
            #maximization(M-step)
            self.transition=self.estimate_transition(alphas,betas)
            self.mu=self.estimate_mu(alphas,betas)
            self.sigma=self.estimate_sigma(alphas,betas)
            self.multinomial=self.estimate_multinomial(alphas,betas)
            
            print('fin')
          
        return self.q
            
            
        
    

In [59]:
import pandas as pd

def read_file(filename):
    """ Load the data
    return:: x array, y array """
    print("Loading data " + filename)
    df = pd.read_csv(filename, delimiter=' ',  header=None)
    return df.values

filename = "EMGaussian.data"
filename_test = "EMGaussian.test"
x = read_file(filename)
x_test = read_file(filename_test)


Loading data EMGaussian.data
Loading data EMGaussian.test


In [60]:
from emgaussian import EMGaussian
from kmean import KMeans
import matplotlib.pyplot as plt
import math
#Initialize parameters with EMGaussian
emg = EMGaussian(x, isotropic=False)
emg.fit(plot_likelihood=False)

pi=emg.pi
mu=emg.mu
sigma=emg.sigmas
transition = np.array([ [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25]])
obs = x
T=x.shape[0]

#HMM
model = HMM(transition, mu,sigma,obs,T,pi)
model.fit()

Kmeans converged after 14 iterations, loss: 3240.593025
Fit Gaussian converged after 5 iterations
[0.21411783 0.29916339 0.25047386 0.23624493] pi
probq 9.383082117570005e-34
transition*alpha 0.24999999999999992
resul 2.3457705293925003e-34
probq 1.163598829043246e-63
transition*alpha 0.24999999999999992
resul 2.908997072608114e-64
probq 0.9999999999999936
transition*alpha 0.24999999999999992
resul 0.2499999999999983
probq 6.419579407690699e-15
transition*alpha 0.24999999999999992
resul 1.6048948519226742e-15
probq 0.9999424576268549
transition*alpha 0.06249999999999998
resul 0.06249640360167841
probq 3.4566249240954324e-17
transition*alpha 0.06249999999999998
resul 2.1603905775596445e-18
probq 5.736825420842769e-05
transition*alpha 0.06249999999999998
resul 3.5855158880267292e-06
probq 1.7411893678305843e-07
transition*alpha 0.06249999999999998
resul 1.0882433548941149e-08
probq 9.366673427282503e-17
transition*alpha 0.015624999999999997
resul 1.4635427230128907e-18
probq 0.9777784213

TypeError: 'int' object is not subscriptable

In [7]:
T

500

In [11]:
range(10,0)

range(10, 0)

In [13]:
for i in range(10,0,-1):
    print(i)

10
9
8
7
6
5
4
3
2
1
